In [1]:
import torch, pickle, argparse, os, warnings, copy, time, mlflow
import numpy as np, pytorch_lightning as pl, matplotlib.pyplot as plt, eagerpy as ep
from models import ConvNet, CConvNet
from data_loader import load_test_data, load_train_data
from foolbox import PyTorchModel
from foolbox.attacks import LinfDeepFoolAttack
from tqdm.notebook import tqdm
from attack_helper import batched_predictions, batched_logits, run_batched_attack_cpu, batched_logits_eps
from mlflow.tracking.artifact_utils import get_artifact_uri
import matplotlib.pyplot as plt

In [15]:
flat_28x28_MLP_run_names = [1697263106, 1697264117, 1697264852, 1697266762, 1697267108, 1697269780, 1697270716,
                            1697271049, 1697271795, 1697274594, 1697275265, 1697278699, 1697282534, 1697282725,
                            1697283628, 1697285267, 1697286841, 1697289516, 1697291360, 1697291756, 1697292309,
                            1697293673, 1697296087, 1697297687, 1697298663, 1697299099, 1697300189, 1697301644,
                            1697303700, 1697304939]

flat_28x28_MLP_data_aug_run_names = [1697314773, 1697315648, 1697316253, 1697319666, 1697321134, 1697323039,
                                     1697381703, 1697387805, 1697392047, 1697395958, 1697423371,
                                     1697326227, 1697327407, 1697329265, 1697330634, 1697334107, 1697335845,
                                     1697438899, 1697441419, 1697445309, 1697454414, 1697470261,
                                     1697339675, 1697340366, 1697342252, 1697344299, 1697346310, 1697348476,
                                     1697493565, 1697500831, 1697507810, 1697517942, 1697522106,
                                     1697351878, 1697352331, 1697353302, 1697355022, 1697356907, 1697357777,
                                     1697556854, 1697557558, 1697562076, 1697563004, 1697575214,
                                     1697359818, 1697360773, 1697363594, 1697364527, 1697368366, 1697373536,
                                     1697581673, 1697587785, 1697595752, 1697602209, 1697615182]

In [17]:
# tracking_uri = 'sqlite:///mlruns/database.db'
# mlflow.set_tracking_uri(tracking_uri)
# df=mlflow.search_runs(experiment_names=['model_training'])
# for run_name in flat_28x28_MLP_data_aug_run_names:
#     run_id=df[df['tags.mlflow.runName']==str(run_name)]['run_id'].values[0]
#     artifact_path = get_artifact_uri(run_id=run_id, tracking_uri=tracking_uri)
#     os.rename(os.path.join(artifact_path, 'LinfProjectedGradientDescentAttack'), os.path.join(artifact_path, 'LinfProjectedGradientDescentAttack_wrong'))